In [1]:
import numpy as np
import pandas as pd

---
### Setting up basic matrices:
The Pauli matrices are the four 2x2 matrices, I, X, Y, Z.
For this example, we have a 4x4 Hamiltonian that is decomposed into IZ, ZI, XX, YY, and ZZ terms (this notation is not matrix multiplication--it is a Kronecker product, which takes two 2x2 matrices and gives a 4x4 matrix).

In [2]:
I = np.eye(2)
X = np.array([[0., 1.],
              [1., 0.]])
Y = np.array([[0., -1j],
              [1j, 0.]])
Z = np.array([[1., 0.],
              [0., -1.]])

In [3]:
IZ = np.kron(I, Z)
print(IZ)

[[ 1.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0.  1.  0.]
 [ 0. -0.  0. -1.]]


In [4]:
ZI = np.kron(Z, I)
print(ZI)

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1. -0.]
 [ 0.  0. -0. -1.]]


In [5]:
XX = np.kron(X, X)
print(XX)

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


In [6]:
YY = np.kron(Y, Y)
print(YY)

[[ 0.+0.j  0.-0.j  0.-0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  1.-0.j  0.-0.j]
 [ 0.+0.j  1.-0.j  0.+0.j  0.-0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]]


In [7]:
ZZ = np.kron(Z, Z)
print(ZZ)

[[ 1.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0. -1. -0.]
 [ 0. -0. -0.  1.]]


----
### Hamiltonian
We consider the Hamiltonian:
$H = IZ + ZI - XX - YY + ZZ$

In [8]:
H = IZ + ZI - XX - YY + ZZ
print(H)

[[ 3.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j -2.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]]


---
### Measuring an observable
An observable is a matrix that we can measure against a state $\vec{\psi}$. The Hamiltonian is an observable. So are the Pauli terms that it is written as a sum of.

To measure some observable matrix $M$ with respect to a state $\vec{\psi}$, we just compute $\vec{\psi}^{\dagger} M \vec{\psi}$. The $\dagger$ means conjugate transpose = transpose the matrix and then take complex conjugates. In this specific example, all the numbers are real, so it happens to be the same as an ordinary matrix transpose.

Here is a helper method that does everything for us:

In [9]:
def expected_value(M, state):
    return np.asmatrix(state).getH() @ M @ state  # .getH() does conjugate tranpose

---
### Let's try for $\vec{\psi} = [0, 1, 0, 0]^T$

In [10]:
state = np.array([[0.], [1.], [0.], [0.]])

#### Method 1: measure $H$ directly
A quantum computer can't actually measure $H$ directly, but let's just check what the answer should be.

In [11]:
print(expected_value(H, state))

[[-1.+0.j]]


#### Method 2: linearity of expectation
A quantum computer can compute the Pauli observables though. So let's use linearity of expectation instead:

In [12]:
IZ_expectation = expected_value(IZ, state)
ZI_expectation = expected_value(ZI, state)
XX_expectation = expected_value(XX, state)
YY_expectation = expected_value(YY, state)
ZZ_expectation = expected_value(ZZ, state)
print(IZ_expectation, ZI_expectation, XX_expectation, YY_expectation, ZZ_expectation)
print(IZ_expectation + ZI_expectation - XX_expectation - YY_expectation + ZZ_expectation)

[[-1.]] [[1.]] [[0.]] [[0.+0.j]] [[-1.]]
[[-1.+0.j]]


Yup, we got -1 both ways!

---
### Variances / Covariances

In [13]:
def variance(M, state):
    # variances is <M^2> - <M>^2
    return expected_value(M @ M, state) - expected_value(M, state) ** 2

def covariance(M_1, M_2, state):
    return expected_value(M_1 @ M_2, state) - expected_value(M_1, state) * expected_value(M_2, state)

The cells below show that:
- Var(H) = 4
- Var(IZ) + Var(ZI) + Var(XX) + Var(YY) + Var(ZZ) = 2 -- not equal because we need to account for covariances
- the summed covariances do indeed equal 4. Note that the XX and ZZ

In [14]:
variance(H, state)

matrix([[4.+0.j]])

In [15]:
variance(IZ, state)

matrix([[0.]])

In [16]:
variance(ZI, state)

matrix([[0.]])

In [17]:
variance(XX, state)

matrix([[1.]])

In [18]:
variance(YY, state)

matrix([[1.+0.j]])

In [19]:
variance(ZZ, state)

matrix([[0.]])

In [20]:
terms, coefficients = [IZ, ZI, XX, YY, ZZ], [1., 1., -1., -1., 1.]
total = np.array([[0.+0.j]])
for i in range(5):
    for j in range(5):
        cov = covariance(terms[i], terms[j], state)
        if cov != 0:
            print(cov, i, j)
            total += (coefficients[i] * coefficients[j] * covariance(terms[i], terms[j], state))
print('Variance from summed covariances is %s' % total)

[[1.]] 2 2
[[1.+0.j]] 2 3
[[1.+0.j]] 3 2
[[1.+0.j]] 3 3
Variance from summed covariances is [[4.+0.j]]


----
### Different State, $\vec{\psi} = [0.43 - 0.16i, -0.49, 0.44i, -0.39 + 0.46i]$

In [21]:
state = np.array([[0.43 - 0.16j], [-0.49], [0.43j], [-0.39 + 0.46j]])

In [22]:
variance(H, state)

matrix([[4.35848816-2.22044605e-16j]])

In [23]:
variance(IZ, state), variance(ZI, state), variance(-XX, state), variance(-YY, state), variance(ZZ, state)

(matrix([[0.95576944+0.j]]),
 matrix([[0.989596+0.j]]),
 matrix([[0.76629724+6.6974204e-18j]]),
 matrix([[0.76629724+0.j]]),
 matrix([[0.97693936+0.j]]))

In [24]:
print(covariance(IZ, ZI, state))
print(covariance(IZ, ZZ, state))
print(covariance(ZI, ZZ, state))
print(covariance(-XX, -YY, state))
print(covariance(-XX, ZZ, state))
print(covariance(-YY, ZZ, state))

[[0.1287768+0.j]]
[[-0.06690672+0.j]]
[[-0.1937784+0.j]]
[[0.08370276-3.3487102e-18j]]
[[0.41059608+1.03528297e-18j]]
[[-0.41059608+6.9388939e-18j]]


In [25]:
# Setting 1 (measure all terms separately) would require a number of state preparations equal to:
5 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936)

22.2744964

In [26]:
# Setting 2 (measure {XX}, {YY, ZZ}, {IZ, ZI}) would require a number of state preparations equal to:
3 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936 + 2*0.08370276 + 2*0.1287768)

14.6395752

In [27]:
# Setting 3 (measure {XX, YY, ZZ}, {IZ, ZI}) would require a number of state preparations equal to:
2 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936 + 2*0.08370276 + 2*0.41059608 + 2*-0.41059608 + 2*0.1287768)

9.7597168

### Programatically finding optimal groupings

In [28]:
# Functions for parsing from strings to matrices
import qutip

def _to_matrix(pauli_char):
    if pauli_char == 'X':
        return qutip.sigmax()
    elif pauli_char == 'Y':
        return qutip.sigmay()
    elif pauli_char == 'Z':
        return qutip.sigmaz()
    return qutip.identity(2)
    
def pauli_string_to_matrix(pauli_string, sign=1):
    if pauli_string[0] == '-':
        sign = -1
        pauli_string = pauli_string[1:]
    matrices = []
    for char in pauli_string:
        matrices.append(_to_matrix(char))
    return sign*qutip.tensor(matrices)

pauli_string_to_matrix('-XX')

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0. -1.]
 [ 0.  0. -1.  0.]
 [ 0. -1.  0.  0.]
 [-1.  0.  0.  0.]]

In [29]:
def variance_grouping(grouping, state):
    """
    Computes the variance of a grouping, via the formula
             Var(A+B) = Var(A) + Var(B) + 2*Cov(A,B)
    Inputs:
       Grouping (as a list, e.g. [-XX, -YY, ZZ])
       State (as a np.array, e.g. np.array([[0.], [1.], [0.], [0.]]))
    """
    running_variance = sum([variance(term, state) for term in grouping])
    grouping_size = len(grouping)
    for i in range(grouping_size):
        for j in range(i):
            running_variance += 2*covariance(grouping[i], grouping[j], state)
    return(np.asscalar(running_variance))

In [30]:
# Credit to: https://stackoverflow.com/questions/19368375/set-partitions-in-python
def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller


In [31]:
def grouping_is_pairwise_commuting(grouping_to_test):
    """
    Computes whether all pairs within a grouping commute (in a general sense).
    Inputs:
        grouping_to_test: a single list of *strings*, e.g. ["-XX", "-YY", "ZZ", "IZ", "ZI"]
    Returns:
        True if all pairs commute; False if not all pairs commute
    """
    evaled_grouping_to_test = [pauli_string_to_matrix(term).full() for term in grouping_to_test]
    running_sum = 0
    for i in range(len(evaled_grouping_to_test)):
        for j in range(i+1, len(evaled_grouping_to_test)):
            # RHS evaluates to 1 if non-commuting, 0 if commuting
            running_sum += not np.array_equal(evaled_grouping_to_test[i] @ evaled_grouping_to_test[j],
                               evaled_grouping_to_test[j] @ evaled_grouping_to_test[i])
    # sum will be 0 iff all pairs commute
    return(running_sum == 0)

def is_valid_partition(partition):
    """
    Computes whether all groupings within a partition are pairwise commuting.
    Inputs:
        partition: a list of lists of strings, e.g. [['-XX', '-YY'], ['ZZ', 'IZ', 'ZI']]
    Outputs:
        1 if all groupings within the partition are pairwise commuting. Else 0.
    """
    return(sum([grouping_is_pairwise_commuting(grouping) for grouping in partition]) == len(partition))

In [32]:
def calculate_expected_state_preps_valid_partitions(term_set, state):
    """
    Computes the expected number of state preparations given every valid partition of a set of terms.
    A valid partition is one in which all groupings are pairwise commuting.
    Inputs:
        term_set: list of *strings*, e.g. ["-XX", "-YY", "ZZ", "IZ", "ZI"]
        state: as a np.array, e.g. np.array([[0.], [1.], [0.], [0.]]))
    """
    evaled_term_set = [pauli_string_to_matrix(term).full() for term in term_set]
    partition_list = []
    expected_state_prep_list = []
    for pp in partition(list(range(len(term_set)))):
        partition_list.append([list(np.array(term_set)[grouping]) for grouping in pp])
        expected_state_prep_list.append(sum([variance_grouping(np.array(evaled_term_set)[indices], state)*len(pp) for indices in pp]))
    partition_df = pd.DataFrame({'partition': partition_list, 'expected_state_preps': expected_state_prep_list}).sort_values(by=["expected_state_preps"])
    # Eliminate invalid partitions that contain groupings that are non-pairwise-computing.
    return(partition_df[partition_df['partition'].apply(lambda partition: is_valid_partition(partition))])

# Aspuru-Guzik example
test_term_set = ["-XX", "-YY", "ZZ", "IZ", "ZI"]
state = np.array([[0], [1], [0], [0]])

partition_list = calculate_expected_state_preps_valid_partitions(test_term_set, state)
partition_list


,partition,expected_state_preps
7,"[[-XX], [-YY, ZZ], [IZ, ZI]]",(6+0j)
12,"[[-YY], [-XX, ZZ], [IZ, ZI]]",(6+0j)
4,"[[-XX], [-YY], [ZZ, IZ, ZI]]",(6+0j)
48,"[[-YY], [-XX, ZZ], [IZ], [ZI]]",(8+0j)
38,"[[-XX], [-YY, ZZ], [IZ], [ZI]]",(8+0j)
34,"[[-XX], [-YY], [IZ], [ZZ, ZI]]",(8+0j)
14,"[[-XX], [-YY], [ZZ], [IZ, ZI]]",(8+0j)
24,"[[-XX], [-YY], [ZZ, IZ], [ZI]]",(8+0j)
2,"[[-XX, -YY], [ZZ, IZ, ZI]]",(8+0j)
5,"[[-XX, -YY, ZZ], [IZ, ZI]]",(8+0j)


In [33]:
# Helper function: print all pairwise covariances across commuting terms
def pairwise_covariances_commuting_terms(term_set, state):
    covariance_list = []
    evaled_term_set = [pauli_string_to_matrix(term).full() for term in term_set]
    for i in range(len(evaled_term_set)):
        for j in range(i+1, len(evaled_term_set)):
            # evaluates to 1 if commuting, 0 if non-commuting
            if np.array_equal(evaled_term_set[i] @ evaled_term_set[j],
                               evaled_term_set[j] @ evaled_term_set[i]):
                covariance_list.append([term_set[i], term_set[j], covariance(evaled_term_set[i], evaled_term_set[j], state).item()])
    return(pd.DataFrame(covariance_list, columns=["term 1", "term 2", "covariance"]))

In [34]:
pairwise_covariances_commuting_terms(test_term_set, state)

,term 1,term 2,covariance
0,-XX,-YY,(1+0j)
1,-XX,ZZ,0j
2,-YY,ZZ,0j
3,ZZ,IZ,0j
4,ZZ,ZI,0j
5,IZ,ZI,0j


In [35]:
state_list = [np.array([[1], [0], [0], [0]]),
              np.array([[0], [1], [0], [0]]),
              np.array([[0], [0], [1], [0]]),
              np.array([[0], [0], [0], [1]])]

In [36]:
for state in state_list:
    print(state)
    print(pairwise_covariances_commuting_terms(test_term_set, state))
    print(calculate_expected_state_preps_valid_partitions(test_term_set, state))

[[1]
 [0]
 [0]
 [0]]
  term 1 term 2  covariance
0    -XX    -YY     (-1+0j)
1    -XX     ZZ          0j
2    -YY     ZZ          0j
3     ZZ     IZ          0j
4     ZZ     ZI          0j
5     IZ     ZI          0j
                           partition  expected_state_preps
47    [[-XX, -YY], [ZZ], [IZ], [ZI]]                    0j
35      [[-XX, -YY, ZZ], [IZ], [ZI]]                    0j
31      [[-XX, -YY], [IZ], [ZZ, ZI]]                    0j
21      [[-XX, -YY], [ZZ, IZ], [ZI]]                    0j
11      [[-XX, -YY], [ZZ], [IZ, ZI]]                    0j
5         [[-XX, -YY, ZZ], [IZ, ZI]]                    0j
2         [[-XX, -YY], [ZZ, IZ, ZI]]                    0j
4       [[-XX], [-YY], [ZZ, IZ, ZI]]                (6+0j)
12      [[-YY], [-XX, ZZ], [IZ, ZI]]                (6+0j)
7       [[-XX], [-YY, ZZ], [IZ, ZI]]                (6+0j)
48    [[-YY], [-XX, ZZ], [IZ], [ZI]]                (8+0j)
38    [[-XX], [-YY, ZZ], [IZ], [ZI]]                (8+0j)
24    [[-XX], [-

In [37]:
# ugly code to calculate the expected state preps per each of the basis states
df0 = calculate_expected_state_preps_valid_partitions(test_term_set, state_list[0])
df1 = calculate_expected_state_preps_valid_partitions(test_term_set, state_list[1])
df2 = calculate_expected_state_preps_valid_partitions(test_term_set, state_list[2])
df3 = calculate_expected_state_preps_valid_partitions(test_term_set, state_list[3])

temp = pd.merge(df0, df1, left_index=True, right_index=True)
temp = pd.merge(temp, df2, left_index=True, right_index=True)
temp = pd.merge(temp, df3, left_index=True, right_index=True)
temp = temp.iloc[:,[0,1,3,5,7]]
temp.columns=["partition", "00", "01", "10", "11"]
temp['row_mean'] = temp.iloc[:,1:].mean(axis=1)
temp['row_var'] = temp.iloc[:,1:].var(axis=1)
temp.sort_values(by=["row_mean"])

/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:36: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial)


,partition,00,01,10,11,row_mean,row_var
5,"[[-XX, -YY, ZZ], [IZ, ZI]]",0j,(8+0j),(8+0j),0j,(4+0j),16.0
2,"[[-XX, -YY], [ZZ, IZ, ZI]]",0j,(8+0j),(8+0j),0j,(4+0j),16.0
35,"[[-XX, -YY, ZZ], [IZ], [ZI]]",0j,(12+0j),(12+0j),0j,(6+0j),36.0
31,"[[-XX, -YY], [IZ], [ZZ, ZI]]",0j,(12+0j),(12+0j),0j,(6+0j),36.0
21,"[[-XX, -YY], [ZZ, IZ], [ZI]]",0j,(12+0j),(12+0j),0j,(6+0j),36.0
11,"[[-XX, -YY], [ZZ], [IZ, ZI]]",0j,(12+0j),(12+0j),0j,(6+0j),36.0
4,"[[-XX], [-YY], [ZZ, IZ, ZI]]",(6+0j),(6+0j),(6+0j),(6+0j),(6+0j),0.0
12,"[[-YY], [-XX, ZZ], [IZ, ZI]]",(6+0j),(6+0j),(6+0j),(6+0j),(6+0j),0.0
7,"[[-XX], [-YY, ZZ], [IZ, ZI]]",(6+0j),(6+0j),(6+0j),(6+0j),(6+0j),0.0
47,"[[-XX, -YY], [ZZ], [IZ], [ZI]]",0j,(16+0j),(16+0j),0j,(8+0j),64.0


In [38]:
# Are the ideal partitions the same with random kets?
random_state_list = [qutip.rand_ket(4).full() for i in range(4)]
print(random_state_list)

# ugly code to calculate the expected state preps per each of the basis states
df0 = calculate_expected_state_preps_valid_partitions(test_term_set, random_state_list[0])
df1 = calculate_expected_state_preps_valid_partitions(test_term_set, random_state_list[1])
df2 = calculate_expected_state_preps_valid_partitions(test_term_set, random_state_list[2])
df3 = calculate_expected_state_preps_valid_partitions(test_term_set, random_state_list[3])

temp = pd.merge(df0, df1, left_index=True, right_index=True)
temp = pd.merge(temp, df2, left_index=True, right_index=True)
temp = pd.merge(temp, df3, left_index=True, right_index=True)
temp = temp.iloc[:,[0,1,3,5,7]]
temp.columns=["partition", "00", "01", "10", "11"]
temp['row_mean'] = temp.iloc[:,1:].mean(axis=1)
temp['row_var'] = temp.iloc[:,1:].var(axis=1)
temp = temp.sort_values(by=["row_mean"]).round(decimals=2)
rounded_temp = temp
rounded_temp.iloc[:,1:] = temp.iloc[:,1:].apply(lambda x: np.around(x, decimals=2), axis=0)
rounded_temp

[array([[ 0.51414978-0.09424261j],
       [-0.02287104-0.05587584j],
       [ 0.302804  -0.5959059j ],
       [ 0.51916538-0.08243996j]]), array([[-0.58208974+0.40000496j],
       [ 0.29272803-0.60477282j],
       [ 0.09988454+0.16019287j],
       [-0.08931179-0.07818202j]]), array([[ 0.49764924-0.30396103j],
       [-0.26873659-0.56848373j],
       [ 0.12769463-0.42569028j],
       [ 0.22999317+0.11893201j]]), array([[0.0305841 -0.51777889j],
       [0.3677753 +0.5048436j ],
       [0.26456331-0.22602001j],
       [0.17026159+0.43677923j]])]


/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:36: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial)


,partition,00,01,10,11,row_mean,row_var
5,"[[-XX, -YY, ZZ], [IZ, ZI]]",(10.42-0j),(6.86+0j),(10.66-0j),(9.71+0j),(9.41-0j),2.29
2,"[[-XX, -YY], [ZZ, IZ, ZI]]",(9.94+0j),(11.75+0j),(10.54-0j),(10.37+0j),(10.65-0j),0.45
12,"[[-YY], [-XX, ZZ], [IZ, ZI]]",(11.12-0j),(11.14+0j),(12.62-0j),(16.17+0j),(12.76-0j),4.25
11,"[[-XX, -YY], [ZZ], [IZ, ZI]]",(14.94+0j),(11.96+0j),(11.93-0j),(14.96+0j),(13.45-0j),2.25
7,"[[-XX], [-YY, ZZ], [IZ, ZI]]",(17.05-0j),(11.62+0j),(14.85+0j),(10.75+0j),(13.57-0j),6.38
35,"[[-XX, -YY, ZZ], [IZ], [ZI]]",(13.86-0j),(10.51+0j),(17.32-0j),(14.28+0j),(13.99-0j),5.82
31,"[[-XX, -YY], [IZ], [ZZ, ZI]]",(16.07+0j),(12.45+0j),(14.23-0j),(13.39+0j),(14.04-0j),1.78
21,"[[-XX, -YY], [ZZ, IZ], [ZI]]",(10.22+0j),(17.57+0j),(16.17-0j),(16.55+0j),(15.13-0j),8.28
4,"[[-XX], [-YY], [ZZ, IZ, ZI]]",(13.7-0j),(17.88+0j),(15.59-0j),(14.26+0j),(15.36-0j),2.59
48,"[[-YY], [-XX, ZZ], [IZ], [ZI]]",(12.46-0j),(15.14+0j),(18.6-0j),(21.18+0j),(16.84-0j),11.00


## Lithium Hydride

### Read in Hamiltonians

In [39]:
def construct_full_term_string(specified_term_list, n):
    """
    Given a list of terms, e.g. ['X1', 'Z3'] 
    Input:
      List of specified terms: e.g. ['X1', 'Z3']
      Number of total terms: e.g. 4
    Outputs:
      String of the full term, e.g. IXIZ
    """
    building_string = ['I']*n # default is that the position should be an I
    for term in specified_term_list:
        split_string = list(term) # e.g. list('X1') becomes ['X', '1']
        building_string[int(split_string[1])] = split_string[0]
    return(''.join(building_string))

In [40]:
from term_grouping import parseHamiltonian
sample_h1 = parseHamiltonian('hamiltonians/sampleH1.txt')
sample_h1_terms = [construct_full_term_string(x[1], 4) for x in sample_h1]
sample_h1_coefs = [x[0] for x in sample_h1]
print(sample_h1_terms)
print(sample_h1_coefs)

lih_jw = parseHamiltonian('hamiltonians/LiH_JW.txt')
lih_jw_terms = [construct_full_term_string(x[1], 4) for x in lih_jw]
lih_jw_coefs = [x[0] for x in lih_jw]
print(lih_jw_terms)
print(lih_jw_coefs)

lih_bk = parseHamiltonian('hamiltonians/LiH_BK.txt')
lih_bk_terms = [construct_full_term_string(x[1], 4) for x in lih_bk]
lih_bk_coefs = [x[0] for x in lih_bk]
print(lih_bk_terms)
print(lih_bk_coefs)

['IIII', 'XXYY', 'XYYX', 'YXXY', 'YYXX', 'ZIII', 'ZZII', 'ZIZI', 'ZIIZ', 'IZII', 'IZZI', 'IZIZ', 'IIZI', 'IIZZ', 'IIIZ']
[2.806800745053043, -0.006811585442824442, 0.006811585442824442, 0.006811585442824442, -0.006811585442824442, -0.007220443762164415, 0.24086324970819822, 0.09808757340260295, 0.10489915884542617, -0.007220443762164414, 0.10489915884542617, 0.09808757340260295, -0.47497738101613074, 0.09255914539024543, -0.47497738101613074]
['IIII', 'XXYY', 'XYYX', 'XZXI', 'XZXZ', 'XIXI', 'YXXY', 'YYXX', 'YZYI', 'YZYZ', 'YIYI', 'ZIII', 'ZXZX', 'ZYZY', 'ZZII', 'ZIZI', 'ZIIZ', 'IXZX', 'IXIX', 'IYZY', 'IYIY', 'IZII', 'IZZI', 'IZIZ', 'IIZI', 'IIZZ', 'IIIZ']
[-7.49894690201071, -0.0029329964409502266, 0.0029329964409502266, 0.01291078027311749, -0.0013743761078958677, 0.011536413200774975, 0.0029329964409502266, -0.0029329964409502266, 0.01291078027311749, -0.0013743761078958677, 0.011536413200774975, 0.16199475388004184, 0.011536413200774975, 0.011536413200774975, 0.12444770133137588, 0.

In [41]:
basis_states_16 = [qutip.basis(16,i).full() for i in range(16)]

In [42]:
# TIMES OUT
# calculate_expected_state_preps_valid_partitions(lih_jw_terms, basis_states_16[0])

### Optimal groupings for LiH_BK

### Optimal groupings for LiH_JW